In this notebook I will perform initial wrangling and cleaning of data for a capstone project investigating correlations between Cascade Care Savings uptake (a Washington state health insurance premium subsidy newly implemented in 2023), area deprivation index, and social determinants of health at the county level.

In [1]:
# import relevant libraries and packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

In [2]:
# import spring 2023 WA state Cascade Care Savings data by county
# Data source: https://www.wahbexchange.org/about-the-exchange/reports-data/enrollment-reports-data/
# See "2023 Spring Enrollment (Excel)"
ccs = pd.read_csv('CCS_uptake_by_county.csv', index_col = 0, thousands = ',')

In [3]:
# view first few lines of data
ccs.head()

,CCS enrollees,Total QHP enrollees,Percent uptake
County,,,
ADAMS,77,294,0.261905
ASOTIN,94,405,0.232099
BENTON,911,3941,0.231160
CHELAN,533,2596,0.205316
CLALLAM,516,2687,0.192036


In [4]:
# View info
ccs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39 entries, ADAMS to YAKIMA
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CCS enrollees        39 non-null     int64  
 1   Total QHP enrollees  39 non-null     int64  
 2   Percent uptake       39 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 1.2+ KB


Great. This spreadsheet contains the index column (county names), CCS enrollee count, total number of people in a qualified health plan, and percent uptake. There are 39 rows for the 39 counties. Dtypes are correct.

In [5]:
# Change county names to title case
ccs.index = ccs.index.str.title()

In [6]:
# Check to ensure it worked
ccs.head()

,CCS enrollees,Total QHP enrollees,Percent uptake
County,,,
Adams,77,294,0.261905
Asotin,94,405,0.232099
Benton,911,3941,0.231160
Chelan,533,2596,0.205316
Clallam,516,2687,0.192036


In [7]:
# Load the next spreadsheet: county FIPS. Check out the data
# Data source: https://www2.census.gov/geo/docs/reference/codes2020/cou/st53_wa_cou2020.txt
fips = pd.read_csv('county_FIPS.csv')
fips.head(6)

,STATE,STATEFP,COUNTYFP,COUNTYNS,COUNTYNAME,CLASSFP,FUNCSTAT
0,WA,53,1,1531601,Adams County,H1,A
1,WA,53,3,1533502,Asotin County,H1,A
2,WA,53,5,1513302,Benton County,H1,A
3,WA,53,7,1531932,Chelan County,H1,A
4,WA,53,9,1531341,Clallam County,H1,A
5,WA,53,11,1531820,Clark County,H1,A


This csv requires more cleaning. FIPS (Federal Information Processing Series) codes are 5 digits in length; the "STATEFP" and "COUNTYFP" fields above will need to be combined to create the final, standardized FIPS code.

In [8]:
fips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   STATE       39 non-null     object
 1   STATEFP     39 non-null     int64 
 2   COUNTYFP    39 non-null     int64 
 3   COUNTYNS    39 non-null     int64 
 4   COUNTYNAME  39 non-null     object
 5   CLASSFP     39 non-null     object
 6   FUNCSTAT    39 non-null     object
dtypes: int64(3), object(4)
memory usage: 2.3+ KB


In [9]:
fips['COUNTYFP'] = fips['COUNTYFP'].astype(str)

In [10]:
# Need to create the correct number of zeros to create the appropriate 5-digit FIPS code
fips['zeros'] = np.where(
    fips['COUNTYFP'].str.len() == 1, '00', '0')

In [11]:
fips.head(6)

,STATE,STATEFP,COUNTYFP,COUNTYNS,COUNTYNAME,CLASSFP,FUNCSTAT,zeros
0,WA,53,1,1531601,Adams County,H1,A,00
1,WA,53,3,1533502,Asotin County,H1,A,00
2,WA,53,5,1513302,Benton County,H1,A,00
3,WA,53,7,1531932,Chelan County,H1,A,00
4,WA,53,9,1531341,Clallam County,H1,A,00
5,WA,53,11,1531820,Clark County,H1,A,0


In [12]:
# Lambda function to create 5-digit county FIPS code
fips['FIPS'] = fips[['STATEFP','zeros','COUNTYFP']].astype(str).apply(lambda x: ''.join(x), axis = 1)
fips.head(6)

,STATE,STATEFP,COUNTYFP,COUNTYNS,COUNTYNAME,CLASSFP,FUNCSTAT,zeros,FIPS
0,WA,53,1,1531601,Adams County,H1,A,00,53001
1,WA,53,3,1533502,Asotin County,H1,A,00,53003
2,WA,53,5,1513302,Benton County,H1,A,00,53005
3,WA,53,7,1531932,Chelan County,H1,A,00,53007
4,WA,53,9,1531341,Clallam County,H1,A,00,53009
5,WA,53,11,1531820,Clark County,H1,A,0,53011


In [13]:
# Clean key column
fips['County'] = fips['COUNTYNAME'].str.replace(' County', '')

In [15]:
# Drop unnecessary columns
fips = fips[['County', 'FIPS']]

In [16]:
fips.head()

,County,FIPS
0,Adams,53001
1,Asotin,53003
2,Benton,53005
3,Chelan,53007
4,Clallam,53009


In [17]:
fips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   County  39 non-null     object
 1   FIPS    39 non-null     object
dtypes: object(2)
memory usage: 752.0+ bytes


In [18]:
fips['FIPS'] = pd.to_numeric(fips['FIPS'])

In [19]:
fips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   County  39 non-null     object
 1   FIPS    39 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 752.0+ bytes


In [20]:
# merge dataframes
merged = pd.merge(ccs, fips, how = 'left', on = ['County','County'])

In [21]:
merged.head()

,County,CCS enrollees,Total QHP enrollees,Percent uptake,FIPS
0,Adams,77,294,0.261905,53001
1,Asotin,94,405,0.232099,53003
2,Benton,911,3941,0.231160,53005
3,Chelan,533,2596,0.205316,53007
4,Clallam,516,2687,0.192036,53009


In [22]:
# Load the next CSV, which contains 2021 area deprivation index by Census block group.
# Data source: https://www.neighborhoodatlas.medicine.wisc.edu/download (need to sign in to access)
adi = pd.read_csv('WA_2021_ADI_Census_Block_Group_v4_0_1.csv')
adi.head()

,GISJOIN,FIPS,ADI_NATRANK,ADI_STATERNK
0,G53000109501001,530019501001,43,8
1,G53000109501002,530019501002,71,10
2,G53000109501003,530019501003,94,10
3,G53000109502001,530019502001,83,10
4,G53000109502002,530019502002,71,10


ADI_STATERNK is a decile that ranks Census block groups from most advantaged (1) to most disadvantaged (10) within the state of Washington. ADI_NATRANK is a percentile that compares Census block groups nationally from most advantaged (1-10) to most disadvantaged (100).

In [23]:
# Rename FIPS column to Census block FIPS
adi['FIPS_Census_block'] = adi['FIPS']
adi = adi.drop(['FIPS'], axis = 1)
adi.head()

,GISJOIN,ADI_NATRANK,ADI_STATERNK,FIPS_Census_block
0,G53000109501001,43,8,530019501001
1,G53000109501002,71,10,530019501002
2,G53000109501003,94,10,530019501003
3,G53000109502001,83,10,530019502001
4,G53000109502002,71,10,530019502002


In [24]:
#Create 5-digit county level FIPS code
adi['County_fips'] = adi['FIPS_Census_block'].astype(str).str[:5]
adi.head()

,GISJOIN,ADI_NATRANK,ADI_STATERNK,FIPS_Census_block,County_fips
0,G53000109501001,43,8,530019501001,53001
1,G53000109501002,71,10,530019501002,53001
2,G53000109501003,94,10,530019501003,53001
3,G53000109502001,83,10,530019502001,53001
4,G53000109502002,71,10,530019502002,53001


In [25]:
# Now can drop FIPS Census block column
adi = adi.drop(['FIPS_Census_block'], axis = 1)
adi.head()

,GISJOIN,ADI_NATRANK,ADI_STATERNK,County_fips
0,G53000109501001,43,8,53001
1,G53000109501002,71,10,53001
2,G53000109501003,94,10,53001
3,G53000109502001,83,10,53001
4,G53000109502002,71,10,53001


In [26]:
adi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5311 entries, 0 to 5310
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   GISJOIN       5311 non-null   object
 1   ADI_NATRANK   5311 non-null   object
 2   ADI_STATERNK  5311 non-null   object
 3   County_fips   5311 non-null   object
dtypes: object(4)
memory usage: 166.1+ KB


In [27]:
adi['ADI_NATRANK'].unique()

array(['43', '71', '94', '83', '86', '46', '84', '59', '55', '56', '57',
       '42', '41', '32', '44', '51', '40', '47', '78', '79', '74', '73',
       '66', '69', '30', '28', '23', '19', '52', '54', '62', '50', '45',
       '70', '49', '61', '25', '13', '21', '38', '14', '29', '22', '18',
       '35', '8', '15', '26', '33', '37', '27', '60', '16', '39', 'GQ',
       '68', '63', '53', '58', '64', '48', '65', '77', '34', 'PH', '9',
       '24', '20', '31', '11', '36', '17', '12', '88', '82', '6', '93',
       '96', '10', '80', '90', '67', '76', '100', '92', '75', '99', '72',
       '98', 'QDI', '85', '7', '3', '4', '2', '5', '1', 'GQ-PH', '97',
       '87', '91', '95', '81', '89'], dtype=object)

Definitions from https://www.neighborhoodatlas.medicine.wisc.edu: \
PH: for suppression due to low population and/or housing \
GQ: for suppression due to a high group quarters population \
PH-GQ: for suppression due to both types of suppression criteria \
QDI: designates block groups without an ADI due to Questionable Data Integrity, stemming from missing data in the source ACS data.

In [28]:
# Next step is to check how common these strings are
gqph = adi.loc[adi['ADI_NATRANK'] == 'GQ-PH']
gqph['GISJOIN'].nunique()

12

In [29]:
ph = adi.loc[adi['ADI_NATRANK'] == 'PH']
ph['GISJOIN'].nunique()

22

In [30]:
gq = adi.loc[adi['ADI_NATRANK'] == 'GQ']
gq['GISJOIN'].nunique()

48

In [31]:
qdi = adi.loc[adi['ADI_NATRANK'] == 'QDI']
qdi['GISJOIN'].nunique()

5

The total number of Census blocks that have string values for ADI_NATRANK is 87. With 5311 unique Census blocks available for analysis, they constitute 1.6% of data.

In [32]:
# Repeat the process for ADI_STATERNK
adi['ADI_STATERNK'].unique()

array(['8', '10', '9', '7', '6', '5', '4', '3', '2', 'GQ', 'PH', '1',
       'QDI', 'GQ-PH'], dtype=object)

In [33]:
gqph_s = adi.loc[adi['ADI_STATERNK'] == 'GQ-PH']
gqph_s['GISJOIN'].nunique()

12

In [34]:
ph_s = adi.loc[adi['ADI_STATERNK'] == 'PH']
ph_s['GISJOIN'].nunique()

22

In [35]:
gq_s = adi.loc[adi['ADI_STATERNK'] == 'GQ']
gq_s['GISJOIN'].nunique()

48

In [36]:
qdi_s = adi.loc[adi['ADI_STATERNK'] == 'QDI']
qdi_s['GISJOIN'].nunique()

5

Looks like Census blocks that lack national ranking data lack state ranking data as well. This makes sense given that these strings are present to mark problems with source data from which the rankings are calculated. From the map at https://www.neighborhoodatlas.medicine.wisc.edu/mapping, some of these problem areas are quite large (e.g., most of Skamania County is not represented) and were suppressed due to low population/low housing. Areas that were suppressed for high group quarters are usually very small. I will drop the affected rows, knowing that other nearby Census blocks will compensate for those small areas dropped due to high group quarters.

In [37]:
# Create list of values to remove & remove them
filter_list = ['GQ-PH','GQ', 'PH', 'QDI']
adi = adi.loc[~adi['ADI_NATRANK'].isin(filter_list)]

In [38]:
adi['ADI_NATRANK'].unique()

array(['43', '71', '94', '83', '86', '46', '84', '59', '55', '56', '57',
       '42', '41', '32', '44', '51', '40', '47', '78', '79', '74', '73',
       '66', '69', '30', '28', '23', '19', '52', '54', '62', '50', '45',
       '70', '49', '61', '25', '13', '21', '38', '14', '29', '22', '18',
       '35', '8', '15', '26', '33', '37', '27', '60', '16', '39', '68',
       '63', '53', '58', '64', '48', '65', '77', '34', '9', '24', '20',
       '31', '11', '36', '17', '12', '88', '82', '6', '93', '96', '10',
       '80', '90', '67', '76', '100', '92', '75', '99', '72', '98', '85',
       '7', '3', '4', '2', '5', '1', '97', '87', '91', '95', '81', '89'],
      dtype=object)

In [39]:
# Make sure this removed all problematic values from STATERNK too
adi['ADI_STATERNK'].unique()

array(['8', '10', '9', '7', '6', '5', '4', '3', '2', '1'], dtype=object)

In [40]:
# Now convert dtype to numeric from object
adi['ADI_NATRANK'] = pd.to_numeric(adi['ADI_NATRANK'])

In [41]:
adi['ADI_STATERNK'] = pd.to_numeric(adi['ADI_STATERNK'])

In [42]:
adi['County_fips'] = pd.to_numeric(adi['County_fips'])

In [43]:
# Drop unnecessary column
adi = adi[['County_fips', 'ADI_NATRANK', 'ADI_STATERNK']]

In [44]:
adi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5224 entries, 0 to 5310
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   County_fips   5224 non-null   int64
 1   ADI_NATRANK   5224 non-null   int64
 2   ADI_STATERNK  5224 non-null   int64
dtypes: int64(3)
memory usage: 163.2 KB


In [45]:
# Need to aggregate Census blocks to the county level to match the CCS uptake data
grouped = adi.groupby(['County_fips'], as_index = False).mean()

In [46]:
grouped.head()

,County_fips,ADI_NATRANK,ADI_STATERNK
0,53001,65.200000,9.466667
1,53003,58.285714,9.142857
2,53005,44.671642,7.858209
3,53007,33.838710,6.483871
4,53009,43.492308,7.784615


In [47]:
# Combine the dataframes
frames = [merged, grouped]
df = pd.concat(frames, axis = 1, join = 'inner')

In [48]:
df = df.drop('County_fips', axis = 1)

In [49]:
df.head()

,County,CCS enrollees,Total QHP enrollees,Percent uptake,FIPS,ADI_NATRANK,ADI_STATERNK
0,Adams,77,294,0.261905,53001,65.200000,9.466667
1,Asotin,94,405,0.232099,53003,58.285714,9.142857
2,Benton,911,3941,0.231160,53005,44.671642,7.858209
3,Chelan,533,2596,0.205316,53007,33.838710,6.483871
4,Clallam,516,2687,0.192036,53009,43.492308,7.784615


In [50]:
# Get social determinants of health dataset. This version contains WA data only by Census tract.
# Source: https://data.cdc.gov/500-Cities-Places/SDOH-Measures-for-County-ACS-2017-2021/i6u4-y3g4/about_data
sdoh = pd.read_csv('SDOH_Measures_for_Census_Tract__ACS_2017-2021_20240223.csv')
sdoh.head()

,Year,StateAbbr,StateDesc,CountyName,CountyFIPS,LocationName,DataSource,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,MOE,TotalPopulation,LocationID,CategoryID,MeasureID,DataValueTypeID,Short_Question_Text,Geolocation
0,2017-2021,WA,Washington,Thurston County,53067,53067010100,5-year ACS,SDOH,Unemployment among people 16 years and older i...,%,Percentage,4.7,2.8,3263,53067010100,SDOH,UNEMP,Percent,Unemployment,POINT (-122.9023537 47.0452755)
1,2017-2021,WA,Washington,Cowlitz County,53015,53015000902,5-year ACS,SDOH,No broadband internet subscription among house...,%,Percentage,8.7,11.3,3601,53015000902,SDOH,BROAD,Percent,No broadband,POINT (-122.9276715 46.1799796)
2,2017-2021,WA,Washington,Lewis County,53041,53041971502,5-year ACS,SDOH,No broadband internet subscription among house...,%,Percentage,9.1,8.4,5142,53041971502,SDOH,BROAD,Percent,No broadband,POINT (-123.1896203 46.504057)
3,2017-2021,WA,Washington,Chelan County,53007,53007960303,5-year ACS,SDOH,No high school diploma among adults aged 25 ye...,%,Percentage,11.9,9.5,1827,53007960303,SDOH,NOHSDP,Percent,No high school diploma,POINT (-120.0562004 47.8541793)
4,2017-2021,WA,Washington,Spokane County,53063,53063013401,5-year ACS,SDOH,Single-parent households,%,Percentage,1.4,1.6,5755,53063013401,SDOH,SNGPNT,Percent,Single-parent households,POINT (-117.3260677 47.6142731)


In [51]:
# Check if there are kinds of data present other than percentages
sdoh['Data_Value_Type'].unique()

array(['Percentage'], dtype=object)

In [52]:
# Get full descriptions of measures and store them
sdoh_measures = sdoh['Measure'].unique()
print(sdoh_measures)

['Unemployment among people 16 years and older in the labor force'
 'No broadband internet subscription among households'
 'No high school diploma among adults aged 25 years or older'
 'Single-parent households' 'Crowding among housing units'
 'Persons living below 150% of the poverty level'
 'Housing cost burden among households' 'Persons aged 65 years or older'
 'Persons of racial or ethnic minority status']


In [53]:
sdoh = sdoh[['CountyFIPS','MeasureID','Data_Value']]
sdoh.head()

,CountyFIPS,MeasureID,Data_Value
0,53067,UNEMP,4.7
1,53015,BROAD,8.7
2,53041,BROAD,9.1
3,53007,NOHSDP,11.9
4,53063,SNGPNT,1.4


In [54]:
sdoh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15930 entries, 0 to 15929
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CountyFIPS  15930 non-null  int64  
 1   MeasureID   15930 non-null  object 
 2   Data_Value  15924 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 373.5+ KB


In [55]:
# Pivot data to prepare to join to other df
pivot = pd.pivot_table(sdoh, values = ['Data_Value'], index = ['CountyFIPS'], columns = ['MeasureID'])

In [56]:
pivot.head()

Data_Value                                                         \
MeasureID       AGE65      BROAD      CROWD      HCOST     NOHSDP     POV150   
CountyFIPS                                                                     
53001       12.171429  18.114286  11.485714  24.171429  32.428571  32.785714   
53003       22.483333  14.666667   2.083333  25.566667   8.833333  27.500000   
53005       14.586364  11.843182   3.088636  22.490909  10.522727  18.088636   
53007       21.447826  13.282609   3.591304  22.243478  13.891304  19.630435   
53009       30.512500  11.195833   1.979167  26.241667   7.433333  21.079167   

                                           
MeasureID     REMNRTY    SNGPNT     UNEMP  
CountyFIPS                                 
53001       63.914286  7.385714  6.514286  
53003       11.816667  4.533333  6.066667  
53005       32.372727  7.443182  5.093182  
53007       29.178261  5.526087  3.952174  
53009       19.983333  4.670833  5.908333

In [57]:
pivot.columns = pivot.columns.droplevel(0)

In [58]:
pivot = pivot.reset_index().rename_axis(None, axis=1)

In [59]:
pivot.head()

,CountyFIPS,AGE65,BROAD,CROWD,HCOST,NOHSDP,POV150,REMNRTY,SNGPNT,UNEMP
0,53001,12.171429,18.114286,11.485714,24.171429,32.428571,32.785714,63.914286,7.385714,6.514286
1,53003,22.483333,14.666667,2.083333,25.566667,8.833333,27.500000,11.816667,4.533333,6.066667
2,53005,14.586364,11.843182,3.088636,22.490909,10.522727,18.088636,32.372727,7.443182,5.093182
3,53007,21.447826,13.282609,3.591304,22.243478,13.891304,19.630435,29.178261,5.526087,3.952174
4,53009,30.512500,11.195833,1.979167,26.241667,7.433333,21.079167,19.983333,4.670833,5.908333


In [60]:
# Combine dataframes
frames = [df,pivot]
df2 = pd.concat(frames, axis = 1, join = 'inner')
df2.head()

,County,CCS enrollees,Total QHP enrollees,Percent uptake,FIPS,ADI_NATRANK,ADI_STATERNK,CountyFIPS,AGE65,BROAD,CROWD,HCOST,NOHSDP,POV150,REMNRTY,SNGPNT,UNEMP
0,Adams,77,294,0.261905,53001,65.200000,9.466667,53001,12.171429,18.114286,11.485714,24.171429,32.428571,32.785714,63.914286,7.385714,6.514286
1,Asotin,94,405,0.232099,53003,58.285714,9.142857,53003,22.483333,14.666667,2.083333,25.566667,8.833333,27.500000,11.816667,4.533333,6.066667
2,Benton,911,3941,0.231160,53005,44.671642,7.858209,53005,14.586364,11.843182,3.088636,22.490909,10.522727,18.088636,32.372727,7.443182,5.093182
3,Chelan,533,2596,0.205316,53007,33.838710,6.483871,53007,21.447826,13.282609,3.591304,22.243478,13.891304,19.630435,29.178261,5.526087,3.952174
4,Clallam,516,2687,0.192036,53009,43.492308,7.784615,53009,30.512500,11.195833,1.979167,26.241667,7.433333,21.079167,19.983333,4.670833,5.908333


In [61]:
# Drop redundant column
df2 = df2.drop('CountyFIPS', axis = 1)

In [62]:
# The "Percent uptake" field is a percentage, like most of the other fields. Multiply by 100 to match others
df2['Percent uptake'] = df2['Percent uptake']*100
df2.head()

,County,CCS enrollees,Total QHP enrollees,Percent uptake,FIPS,ADI_NATRANK,ADI_STATERNK,AGE65,BROAD,CROWD,HCOST,NOHSDP,POV150,REMNRTY,SNGPNT,UNEMP
0,Adams,77,294,26.190476,53001,65.200000,9.466667,12.171429,18.114286,11.485714,24.171429,32.428571,32.785714,63.914286,7.385714,6.514286
1,Asotin,94,405,23.209877,53003,58.285714,9.142857,22.483333,14.666667,2.083333,25.566667,8.833333,27.500000,11.816667,4.533333,6.066667
2,Benton,911,3941,23.115960,53005,44.671642,7.858209,14.586364,11.843182,3.088636,22.490909,10.522727,18.088636,32.372727,7.443182,5.093182
3,Chelan,533,2596,20.531587,53007,33.838710,6.483871,21.447826,13.282609,3.591304,22.243478,13.891304,19.630435,29.178261,5.526087,3.952174
4,Clallam,516,2687,19.203573,53009,43.492308,7.784615,30.512500,11.195833,1.979167,26.241667,7.433333,21.079167,19.983333,4.670833,5.908333


In [63]:
# Double check dtypes before getting into analysis
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39 entries, 0 to 38
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   County               39 non-null     object 
 1   CCS enrollees        39 non-null     int64  
 2   Total QHP enrollees  39 non-null     int64  
 3   Percent uptake       39 non-null     float64
 4   FIPS                 39 non-null     int64  
 5   ADI_NATRANK          39 non-null     float64
 6   ADI_STATERNK         39 non-null     float64
 7   AGE65                39 non-null     float64
 8   BROAD                39 non-null     float64
 9   CROWD                39 non-null     float64
 10  HCOST                39 non-null     float64
 11  NOHSDP               39 non-null     float64
 12  POV150               39 non-null     float64
 13  REMNRTY              39 non-null     float64
 14  SNGPNT               39 non-null     float64
 15  UNEMP                39 non-null     float